# Test Load Daily

Start by loading one "new" tile.

## Imports

In [1]:
import os
import numpy as np
from astropy.io import fits
from astropy.table import Table
from sqlalchemy.exc import IntegrityError
from desiutil.log import get_logger, DEBUG
from desispec.io.meta import findfile
import specprodDB.load as db
import specprodDB.tile as t

## Initial Values

In [2]:
specprod = os.environ['SPECPROD']
# tile_id, tile_survey, tile_program = 3867, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5053, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5052, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5074, 'main', 'dark'
# tile_id, tile_survey, tile_program = 1685, 'main', 'dark'
# tile_id, tile_survey, tile_program = 40069, 'main', 'backup'
# tile_id, tile_survey, tile_program = 80950, 'sv1', 'backup'
overwrite = True

## Initialize Database

In [3]:
db.log = get_logger(DEBUG)
postgresql = db.setup_db(schema=specprod, hostname='db-loadbalancer.bweaver.development.svc.spin.nersc.org', username='desi_admin', overwrite=overwrite)
if overwrite:
    db.load_versions('computed', 'daily/v0', 'daily', specprod, 'main')

INFO:load.py:1794:setup_db: Begin creating tables.
INFO:load.py:1797:setup_db: Finished creating tables.
INFO:load.py:1540:load_versions: Loading version metadata.
INFO:load.py:1551:load_versions: Completed loading version metadata.


## Read tiles file

In [4]:
# tiles_file = findfile('tiles', readonly=True).replace('.fits', '.csv')
tiles_file = os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'tiles-daily-patched-with-jura.csv')
tiles_table = Table.read(tiles_file, format='ascii.csv')
# tiles_table

In [5]:
row_index = np.where((tiles_table['LASTNIGHT'] >= 20201214) & (tiles_table['EFFTIME_SPEC'] > 0) & (~tiles_table['PROGRAM'].mask))[0]
# tiles_table[row_index]
# row_index = np.where((tiles_table['LASTNIGHT'] >= 20201214) & (tiles_table['EFFTIME_SPEC'] > 0))[0]
# tiles_table[row_index][tiles_table[row_index]['PROGRAM'].mask]

In [6]:
%%time
candidate_tiles = db.Tile.convert(tiles_table, row_index=row_index)

CPU times: user 636 ms, sys: 44 ms, total: 680 ms
Wall time: 687 ms


## Read exposures file

The daily exposures file may contain exposures with `EFFTIME_SPEC == 0`. We do not want to load these. There are also cases where a *tile* has non-zero `EFFTIME_SPEC` but has no *exposures* with non-zero `EFFTIME_SPEC`. At least for now, don't try to load those either.

In [7]:
# exposures_file = findfile('exposures', readonly=True)
exposures_file = os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'exposures-daily-patched-with-jura.fits')
exposures_table = Table.read(exposures_file, format='fits', hdu='EXPOSURES')
frames_table = Table.read(exposures_file, format='fits', hdu='FRAMES')
# exposures_table[exposures_table['TILEID'] == new_tile.tileid]

In [8]:
%%time
load_tiles = list()
bad_tiles = list()
load_exposures = list()
for new_tile in candidate_tiles:
    row_index = np.where((exposures_table['TILEID'] == new_tile.tileid) & (exposures_table['EFFTIME_SPEC'] > 0))[0]
    if len(row_index) > 0:
        load_tiles.append(new_tile)
        load_exposures += db.Exposure.convert(exposures_table, row_index=row_index)
        # if (exposures_table[row_index]['MJD'] < 50000).any():
        #     print("WARNING: Invalid MJD values detected for tile {0:d}!".format(new_tile.tileid))
        #     bad_index = np.where((exposures_table['TILEID'] == new_tile.tileid) & (exposures_table['EFFTIME_SPEC'] > 0) & (exposures_table['MJD'] < 50000))[0]
        #     for row in exposures_table[['EXPID', 'NIGHT', 'MJD', 'EFFTIME_SPEC']][bad_index]:
        #         raw_data_file = os.path.join(os.environ['DESI_SPECTRO_DATA'], "{0:08d}".format(row['NIGHT']), "{0:08d}".format(row['EXPID']), "desi-{0:08d}.fits.fz".format(row['EXPID']))
        #         with fits.open(raw_data_file, mode='readonly') as hdulist:
        #             mjd_obs = hdulist['SPEC'].header['MJD-OBS']
        #             print("WARNING: tile {0:d} exposure {1:d} has MJD-OBS = {2:f} in {3}!".format(new_tile.tileid, row['EXPID'], mjd_obs, raw_data_file))
    else:
        print("ERROR: No valid exposures found for tile {0:d}, even though EFFTIME_SPEC == {1:f}!".format(new_tile.tileid, new_tile.efftime_spec))
        bad_index = np.where((exposures_table['TILEID'] == new_tile.tileid))[0]
        print(exposures_table[['EXPID', 'NIGHT', 'MJD', 'EFFTIME_SPEC']][bad_index])
        bad_tiles.append(new_tile)
# load_exposures

ERROR: No valid exposures found for tile 81088, even though EFFTIME_SPEC == 24.800000!
EXPID  NIGHT   MJD EFFTIME_SPEC
----- -------- --- ------------
83416 20210404 0.0          0.0
ERROR: No valid exposures found for tile 158, even though EFFTIME_SPEC == 60.500000!
EXPID  NIGHT   MJD EFFTIME_SPEC
----- -------- --- ------------
83543 20210405 0.0          0.0
ERROR: No valid exposures found for tile 23, even though EFFTIME_SPEC == 66.700000!
EXPID  NIGHT   MJD EFFTIME_SPEC
----- -------- --- ------------
83524 20210405 0.0          0.0
ERROR: No valid exposures found for tile 50, even though EFFTIME_SPEC == 73.800000!
EXPID  NIGHT   MJD EFFTIME_SPEC
----- -------- --- ------------
83525 20210405 0.0          0.0
ERROR: No valid exposures found for tile 131, even though EFFTIME_SPEC == 79.300000!
EXPID  NIGHT   MJD EFFTIME_SPEC
----- -------- --- ------------
83732 20210406 0.0          0.0
ERROR: No valid exposures found for tile 77, even though EFFTIME_SPEC == 68.300000!
EXPID  NIGH

In [9]:
%%time
load_frames = list()
for exposure in load_exposures:
    row_index = np.where(frames_table['EXPID'] == exposure.expid)[0]
    assert len(row_index) > 0
    load_frames += db.Frame.convert(frames_table, row_index=row_index)
# load_frames

CPU times: user 5min 4s, sys: 5.69 s, total: 5min 10s
Wall time: 5min 17s


In [10]:
%%time
try:
    db.dbSession.add_all(load_tiles)
    db.dbSession.commit()
except IntegrityError:
    print("ERROR: Some tiles are already loaded!")
    db.dbSession.rollback()

CPU times: user 1.46 s, sys: 52 ms, total: 1.51 s
Wall time: 2.38 s


In [11]:
%%time
# db.dbSession.rollback()
db.dbSession.add_all(load_exposures)
db.dbSession.commit()

IntegrityError: (psycopg2.errors.NotNullViolation) null value in column "fiberfac_gfa" violates not-null constraint
DETAIL:  Failing row contains (20201222, 69455, 80610, 0, 0, 2020-12-23 12:02:33.591552+00, 59206.50177768, cmx, dark, elg, cmxelg, 900.004028320312, 1.34727168977261, 4000, dark, 0.9, 1.17609, 0.0214956291019917, 0, 0, 0.156659, 0.0387605, 0.103173, 0.166885976066826, 9.49855, 0, 0, 0, 1.25361, 1.34727, 1.3298, 1.909135184888, 0, 0, 0, 0.0691387515196852, 1.425028437377, 0.421895887401468, 0.319807524199896, 0.151348426191788, null, null, null, 1.18246672966859, 20.5068374559146, 21.6948676964909, 20.7336748369694, 19.1617744587365, null, null, null, null).

[SQL: INSERT INTO daily.exposure (night, expid, tileid, tilera, tiledec, date_obs, mjd, survey, program, faprgrm, faflavor, exptime, efftime_spec, goaltime, goaltype, mintfrac, airmass, ebv, seeing_etc, efftime_etc, tsnr2_elg, tsnr2_qso, tsnr2_lrg, tsnr2_l ... 738229 characters truncated ... ime_gfa__627)s, %(efftime

In [12]:
%%time
# db.dbSession.rollback()
db.dbSession.add_all(load_frames)
db.dbSession.commit()

PendingRollbackError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (psycopg2.errors.NotNullViolation) null value in column "fiberfac_gfa" violates not-null constraint
DETAIL:  Failing row contains (20201222, 69455, 80610, 0, 0, 2020-12-23 12:02:33.591552+00, 59206.50177768, cmx, dark, elg, cmxelg, 900.004028320312, 1.34727168977261, 4000, dark, 0.9, 1.17609, 0.0214956291019917, 0, 0, 0.156659, 0.0387605, 0.103173, 0.166885976066826, 9.49855, 0, 0, 0, 1.25361, 1.34727, 1.3298, 1.909135184888, 0, 0, 0, 0.0691387515196852, 1.425028437377, 0.421895887401468, 0.319807524199896, 0.151348426191788, null, null, null, 1.18246672966859, 20.5068374559146, 21.6948676964909, 20.7336748369694, 19.1617744587365, null, null, null, null).

[SQL: INSERT INTO daily.exposure (night, expid, tileid, tilera, tiledec, date_obs, mjd, survey, program, faprgrm, faflav

## Load photometry for the tile

When tractor photometry is written out by John Moustakas' VAC code, only objects with `brickname != ''` are written.

In [13]:
tile_index = 0

In [14]:
%%time
potential_targets_table = t.potential_targets(load_tiles[tile_index].tileid)
# potential_targets_table

PendingRollbackError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (psycopg2.errors.NotNullViolation) null value in column "fiberfac_gfa" violates not-null constraint
DETAIL:  Failing row contains (20201222, 69455, 80610, 0, 0, 2020-12-23 12:02:33.591552+00, 59206.50177768, cmx, dark, elg, cmxelg, 900.004028320312, 1.34727168977261, 4000, dark, 0.9, 1.17609, 0.0214956291019917, 0, 0, 0.156659, 0.0387605, 0.103173, 0.166885976066826, 9.49855, 0, 0, 0, 1.25361, 1.34727, 1.3298, 1.909135184888, 0, 0, 0, 0.0691387515196852, 1.425028437377, 0.421895887401468, 0.319807524199896, 0.151348426191788, null, null, null, 1.18246672966859, 20.5068374559146, 21.6948676964909, 20.7336748369694, 19.1617744587365, null, null, null, null).

[SQL: INSERT INTO daily.exposure (night, expid, tileid, tilera, tiledec, date_obs, mjd, survey, program, faprgrm, faflav

In [15]:
%%time
potential_cat = t.potential_photometry(load_tiles[tile_index], potential_targets_table)
# potential_cat

NameError: name 'potential_targets_table' is not defined

In [16]:
%%time
potential_targetphot = t.targetphot(potential_cat)

NameError: name 'potential_cat' is not defined

In [17]:
%%time
potential_tractorphot = t.tractorphot(potential_cat)

NameError: name 'potential_cat' is not defined

In [18]:
%%time
load_photometry = t.load_photometry(potential_tractorphot)

NameError: name 'potential_tractorphot' is not defined

### Load photometry, such as it is, for objects that are not in the tractor catalog

In [19]:
%%time
load_targetphot = t.load_targetphot(potential_targetphot, load_photometry)

NameError: name 'potential_targetphot' is not defined

### Load the target table

In [20]:
%%time
load_target = t.load_target(load_tiles[tile_index], potential_targetphot)

NameError: name 'potential_targetphot' is not defined

## Load tile/cumulative redshifts

Need a way to compute "best" spectra as new tiles are added. There are a lot of columns that come from other sources here, so need to track these down.

In [21]:
%%time
load_ztile = t.load_redshift(load_tiles[tile_index])

PendingRollbackError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (psycopg2.errors.NotNullViolation) null value in column "fiberfac_gfa" violates not-null constraint
DETAIL:  Failing row contains (20201222, 69455, 80610, 0, 0, 2020-12-23 12:02:33.591552+00, 59206.50177768, cmx, dark, elg, cmxelg, 900.004028320312, 1.34727168977261, 4000, dark, 0.9, 1.17609, 0.0214956291019917, 0, 0, 0.156659, 0.0387605, 0.103173, 0.166885976066826, 9.49855, 0, 0, 0, 1.25361, 1.34727, 1.3298, 1.909135184888, 0, 0, 0, 0.0691387515196852, 1.425028437377, 0.421895887401468, 0.319807524199896, 0.151348426191788, null, null, null, 1.18246672966859, 20.5068374559146, 21.6948676964909, 20.7336748369694, 19.1617744587365, null, null, null, null).

[SQL: INSERT INTO daily.exposure (night, expid, tileid, tilera, tiledec, date_obs, mjd, survey, program, faprgrm, faflav

## Load fiberassign and potential

In [22]:
%%time
load_fiberassign, load_potential = t.load_fiberassign(load_tiles[tile_index])

PendingRollbackError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (psycopg2.errors.NotNullViolation) null value in column "fiberfac_gfa" violates not-null constraint
DETAIL:  Failing row contains (20201222, 69455, 80610, 0, 0, 2020-12-23 12:02:33.591552+00, 59206.50177768, cmx, dark, elg, cmxelg, 900.004028320312, 1.34727168977261, 4000, dark, 0.9, 1.17609, 0.0214956291019917, 0, 0, 0.156659, 0.0387605, 0.103173, 0.166885976066826, 9.49855, 0, 0, 0, 1.25361, 1.34727, 1.3298, 1.909135184888, 0, 0, 0, 0.0691387515196852, 1.425028437377, 0.421895887401468, 0.319807524199896, 0.151348426191788, null, null, null, 1.18246672966859, 20.5068374559146, 21.6948676964909, 20.7336748369694, 19.1617744587365, null, null, null, null).

[SQL: INSERT INTO daily.exposure (night, expid, tileid, tilera, tiledec, date_obs, mjd, survey, program, faprgrm, faflav

## Recompute global values

The global values are the primary classification and number of spectra.

In [23]:
t.update_primary()

PendingRollbackError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (psycopg2.errors.NotNullViolation) null value in column "fiberfac_gfa" violates not-null constraint
DETAIL:  Failing row contains (20201222, 69455, 80610, 0, 0, 2020-12-23 12:02:33.591552+00, 59206.50177768, cmx, dark, elg, cmxelg, 900.004028320312, 1.34727168977261, 4000, dark, 0.9, 1.17609, 0.0214956291019917, 0, 0, 0.156659, 0.0387605, 0.103173, 0.166885976066826, 9.49855, 0, 0, 0, 1.25361, 1.34727, 1.3298, 1.909135184888, 0, 0, 0, 0.0691387515196852, 1.425028437377, 0.421895887401468, 0.319807524199896, 0.151348426191788, null, null, null, 1.18246672966859, 20.5068374559146, 21.6948676964909, 20.7336748369694, 19.1617744587365, null, null, null, null).

[SQL: INSERT INTO daily.exposure (night, expid, tileid, tilera, tiledec, date_obs, mjd, survey, program, faprgrm, faflav

## q3c Update

`tile`, `exposure`, `photometry`, `fiberassign`

In [ ]:
t.update_q3c()